# Focuses on Allocating the correct CPU and memory, resource for your spark job.

There are logics and rules needed to be followed, in other to optimally size your spark cluster resources, before we can say we need X executors with Y amount of Memory for executors and driver, and Z amount of memory each.

This tutorial answers, how we need to do that.
- How many executors, memories and cpu cores 


## Cluster Resources:
Say we have 5 nodes (machine) in a cluster with:
- Each with 12 CPU cores and 48GB RAM.

How do we decide the number of executor, core and memory.

“Thin vs. Optimal vs. Fat Executors” approach is one way practitioners think about allocating executors, cores, and memory in a cluster.

It’s not an official Spark term (you won’t see it in the docs), but it’s a widely used heuristic in tuning discussions.


**There are 3 ways of deciding, which are:**
- Thin Executors
- Optimal Executors
- Fat Executors

All 3 of them has advantages and disadvantages.

> Leave 1 CPU and 1GB for operation system

1. **Fat Executors:** They occupy larger portions of resources available in a single node.
- Idea: Fewer executors with many cores and large memory.

Given the above resources (5 nodes, 12 cpus and 48gb on each node), so one fat executor is achieved by specifying 11 out of 12 cores and 47GB out of 48GB RAM on each nodes.

`spark-submit 
--num-executor 5 ...
--executo-cores 11
--exec...memory 47g
`
So, 5 nodes (machines) will have 5 execs with 11 CPU and 47GB each.


#### Truth check for Fact executors:
- ✔ True: 11 cores, 47GB → 5 executors = fat executor design.
- ❌ Not exactly true: 5 cores, 25GB → still big, but no longer a fat executor in the strict sense — it’s a mid/optimal setup.
- 💡 Fat executors = max resources per executor, but not always the best choice for ETL workloads.


---



2. **Thin executors:** opposite of fat, occupy min, resources.
- Idea: Use many executors with fewer cores each (e.g., 1–3 cores per executor).

1 executor can have 1 core this means it is possible we run 11 executors on each node of 11 CORES.
if executor core is 1, num of executors is 11 then the number of memory per executor, will be 47GB/11 approx. 4GB.

- 1 node = 11 execs and 4GB.
- 5 nodes = 55 execs, 4GB

So, we will submit our job to a 5 node cluster of 11 CPU core and 47GB each with:

`spark-submit
--num-executor 55
--exec..core 1
--exec..mem... 4g
`


## Advantages of Fat:
- increase task parallelism
- Fault tolerance 

## Disadvantages
...


## Advantages of Thin:
...


## Disadvantages of Thin:
...

## 3. Optimal Executor Sizing:
- Idea: 4–5 cores per executor is often the balance between parallelism and GC efficiency.

**Rules:**
- per node, leave out 1 core and 1GB of ram for Hadoop / yarn / os
- Leave out 1 executor or 1 core or 1gb at cluster level.
- 3 - 5 cores per executor (good practice)
- when you define executor memory it should exclude overhead memory.

### Resources:
Say we have 5 nodes (machine) in a cluster with:
- 12 CPU cores and 48GB RAM each.

Let's begin to configure our optimal executors:

1. Leave out 1 core and 1gb per node (machine):

**Total node-level resource:**
`12 core - 1 core = 11 cpu core.`

**Total node-level after applying rule 1:**
`48gb - 1gb = 47gb`

So we are left with 11 cores and 47gb per node.


2. Leave out either 1 executor or 1 core or 1 gb for the application at the cluster level.

Our total cluster level resource is:

-> `total_memory_in_cluster = 5 nodes x 47GB = 235GB`

-> `total_cores_in_cluster = 5 nodes x 11 cores = 55 cores`

Based on rule 2, subtract 1 CPU and 1 GB, 
`235GB - 1GB = 234GB`
`55 cores - 1 core = 54 cores` 

`Total cluster resuource = 234 GB and 54 cores at cluster level.`

Next, we find the number of executors and memory & core per executor for our Job.

1. Assign 3 - 5 cores:
going by rule 3, we can choose to use 5 cores per executors.

**Find Number of Executors:**
`total_executors = 54 cores / 5 cores -> ~ 10 execs.`
So in total we would have 10 execs and each node.


**Find amount of Memory:**
`memory_per_exec = 234 GB / 10 Execs ~ 23.4`, each executor will have ~ 23 GB.

Let's apply Rule 4. to get the actual exec. memory.

1. Subtract overhead memory, from executor memory.
Over head memory should be max of 384MB or 10% of executor memory. max(384MB, 0.10 * memory_per_exec)

`actual_mem = 23GB - max(384MB or 10% * 23GB)`
10% of 23GB is = 2.3GB
`actual_mem = max(384MB, 2.3GB) = 2.3GB`

:. 23GB - 2.3GB appr ->  20GB
Actual memory per executor = 20GB

In summary the Job configuration will be:

`number of executors -> 10`
`number of cpu per executor -> 5`
`number of memory per executor -> 20 GB`

# What about the size of DATA?
E.G., 10GB, 100GB of data, wouldn't the size of data affect the calculation that has been done above?


**Answer:**
We have previously calculated the memory per core, we can get this by `executor memory` / `executor cores`.

memory_per_core = `20GB / 5 cores = 4GB`

:. Memory per 1 core in an executor is 4GB.

1 core will hold 4GB of partition, since 1 core processes 1 task/partition, this means, as long as the partition size per core is <= 4GB, 1 core in an executor can process it easily.

So the answer revolves around the size of your set partition, default is 128MB, then by this configuration we have made, it will process data seamlessly because 128MB < 4GB. 

You need to know or control your partition size and your `memory per core` in other to process your data size.

💡 Rule of thumb: Aim for ~128 MB (sometimes 200–256 MB for large clusters) per partition after compression.
That’s why you "need to know your partition size."


Putting it together

When tuning:

- Estimate memory per core → decide the maximum safe partition size.
- Control partition count (repartition() / coalesce()) so total data size / partition size = number of partitions.
- Match number of partitions to cluster parallelism (often 2–4× total cores).

# Practical 2: Optimal Executor Sizing
**Using:**
3 Nodes, each has 16 cores and 48GB.

Calculate the number of executors and cores & memory per executor.

**Rule 1: Node level**
Remove 1GB and 1 core from each node.
remaining: 15 cores and 47GB per executor.

**Rule2: Cluster level**
cluster resource: 15 x 3 nodes = 45 cores and 47GB x 3 nodes = 141GB
Remove 1GB and 1 core for OS operations.

remaining: 44 cores and 140GB per executor.

**Rule 3:** use 3 - 5 cores per, using 4 cores.
num_of_execs = 44 / 4 = 11 execs.
number_of_execs_mem = 140 / 11 execs -> aprox. 12GB per execs

**Rule 4:** Exclude overhead memory (10% of exec. mem.)
:. 10% of 12 = 1.2GB 
12GB - 1.2 (aprrox 1 GB) = 12 - 1 = 11 GB

final memory per executor, after excluding overhead memory is:
11GB

**Optimal Executor:**
--num-exec 11
--num-exec-mem 11g
--num-core 4

## Summary:

- Start with executor sizing based on hardware, not data size
- Then adjust parallelism (partitions) based on data size.
- There are many other important Spark optimizations — sizing is step 1.

| Area                   | Key Techniques                                                                        |
| ---------------------- | ------------------------------------------------------------------------------------- |
| **Partitioning**       | Repartition/coalesce wisely, aim for \~100–200MB per partition                        |
| **Join Strategy**      | Use `broadcast()` for small tables, understand physical plans (`SortMergeJoin`, etc.) |
| **Caching**            | Use `.cache()` or `.persist()` when reusing expensive results                         |
| **Shuffle Reduction**  | Avoid wide transformations where possible (groupBy, join, etc.)                       |
| **File Format**        | Use **Parquet** or **ORC** instead of CSV for big data                                |
| **Predicate Pushdown** | Use filters early so they are pushed into file scan                                   |
| **Data Skew**          | Handle skewed keys in joins or aggregations (e.g., salting)                           |
| **Cluster Tuning**     | Use dynamic allocation, fine-tune GC (`spark.executor.extraJavaOptions`)              |


# Shuffle Partition
After sizing your cluster, the next step is adjusting parallelism (partitions) based on data size.

Shuffling happens when we do a wide transformation. Spark tries to bring related data across executors into same partition.
## **Data Shuffling:**
- This is the process of redistributing data across partitions, and typically involves data exchange between executor nodes.
- Wide transformations, require shuffling
- It requires saving data to disk, sending it over network and reading data from the disk
- Data Shuffling can be very expensive
- Sometimes it can be mitigated or avoided by even code changes, like for instance avoiding sorting the data
- Nevertheless, it's often a "necessary evil".


**Step:**
- Once a file is read in spark it is broken into partitions. p1, p2, p3, p4.
- say a shuffle was done (groupBy) in the partition, it will end up moving related data to a single partition, and at the end each partition that holds data is called a `shuffle partition`. 
- By default spark creates 200 shufflue partitions.
- Say you have a cluster of 1000 cores, and after spark reads your data into 200 partitions, this means, 1 core handles 1 partition, which then says 200 cores will process 200, partitions, and this leaves 800 cores idle.
  - This will result to slow completion time and underutilization of cluster resource.



## How to set shuffle partition.
Scenerio 1: Data per shuffle partition is very large, and we tune it to a reasonable number by calculation a shuffle partition size.

Calculation: 
5 execs and 4 cores; default shuffle partition -> 200 parts; data size to be shuffled 300GB.

total_core = 5 x 4 = 20 core;
shuffle_partition = 200 parts

> Find size data per shuffle parts. = 300 GB / 200 s.p = 1.5gb
> this means each core will process 1.5gb of data.

But optimal partition size should be between 1MB - 200MB per core. Which mean 1 executor should process 1 - 200MB of data per task.

Now we need to tune the number of shuffle partition `spark.sql.shuffle.partition` in other for each core st handle an adequate amount of data.

- let's choose 200MB for the number of shuffle partition size, meaning we want 1 cpu to handle a total of 200MB of shuffle partition.
(300 GB x 1000 MB) / 200 MB = 1500 S.P

Therefore,
`spark.sql.shuffle.partition=1500`, this makes 1 cpu process, 200MB.


---


Scenerio 2: Data per shuffle partition is very small, and we tune it to a reasonable number by calculation a shuffle partition size.

Say we have 3 executors, 4 cpu = 12 cores in total;
to process 50MB of data using default S.P of 200 parts.

Partition size per core = 50/200 -> 250KB; 250kb shared across 12 cores.

This is really small and the optimal size is supposed to be within 1MB - 200MB.

op 1: change the number of S.P: choose say 10 MB SP.
so 50MB / 10MB = 5. so set S.P to 5;
But the issue here is, only 5 core out of 12 will process data, this is underutilization even for small data.

set `spark.sql.shuffle.partition=5`

option 2: 50MB / 12 cores = 4.2MB
which means each core will process 4.2MB of data.

set `spark.sql.shuffle.partition=12` 

tuning your executors may not fully solve your slow data processing, there could be issues like Skew (which can be solved with salting or enabling AQE).

> spark.sql.shuffle.partitions = 12 sets the number of partitions after Spark performs a shuffle, not the initial partitions from reading the data ✅.


**Question:**
But can I control the initial read shuffle partition?

**Answer:**
✅ Yes, you can control the number of partitions when reading data — but it's separate from shuffle partitions `spark.sql.shuffle.partition`.

> df = spark.read.option("maxPartitionBytes", value).csv("path")
or
> spark.conf.set("spark.sql.files.maxPartitionBytes", "128MB") 
This controls how Spark splits the input files into initial partitions (before any transformations or shuffles).


You can also manually repartition:
> df = spark.read.csv("path").repartition(12)
12 initial partitions, before any shuffle.
